In [ ]:
import numpy as np
import pandas as pd
#from pymatgen.core import structure, Lattice
import re
from scipy.spatial.transform import Rotation
import open3d as o3d
from tqdm.auto import tqdm


def get_grid_points(grid):
    dims = grid
    return [np.array([i, j, k]) for i in np.linspace(0, 1, dims[0]) for j in np.linspace(0, 1, dims[1]) for k in np.linspace(0, 1, dims[2])]

def grid_points_to_cartesian(lattice, grid_points):
    return [lattice.get_cartesian_coords(gp) for gp in grid_points]


In [ ]:
import json
with open("/path/to/saved/icsd_id_struc.json", "r") as f:
    dic = json.load(f)

In [ ]:
def make_np(i, dic):
    data = pd.read_csv(i)
    lp = re.findall("[+-]?\d+\.\d+", data.iloc[0, 0])
    if len(lp) == 0:
        lp = re.findall("[+-]?\d+\.\d+", data.columns.values[0])
    if any(i in ['-1.#IND00E+00', '1.#IND00E+00', '-1.#INF00E+00',  '1.#INF00E+00', '-1.#INF00E+00'] for i in data.loc[10][0].split('  ')):
        pass
    else:
        grid = np.array([int(s) for s in re.findall(r'\d+', data.loc[1][0])])

        if grid.size != 3:
            grid = np.array([int(s) for s in re.findall(r'\d+', data.loc[0][0])])

            data_len = len(data.loc[1:])
        else:

            data_len = len(data.loc[2:])

        #lattice = Lattice.from_parameters(float(lp[0]),float(lp[1]),float(lp[2]), float(lp[3]), float(lp[4]), float(lp[5]))
        grid_points = get_grid_points(grid)
        
        cartesian_points = np.dot(grid_points, dic[i.split('/')[-1].split('.')[0]])


        tar = [] 

        for k in  range(data_len):
            if data_len == len(data.loc[2:]):
                stringlist = data.loc[k + 2][0].split("  ")
            else:
                 stringlist = data.loc[k + 1][0].split("  ")
            res = [val for idx, val in enumerate(stringlist)
               if val or (not val and stringlist[idx])]
            try:
                res = np.array(res, dtype=np.float16)
            except ValueError:
                break
            else:
                tar.extend(res)


        if len(tar) != grid[0]*grid[1]*grid[2]:
            pass
        else:
            pc = np.zeros((len(tar), 4), dtype = np.float16)

            pc[:, :3] = np.array(cartesian_points, dtype = np.float16)
            pc[:, 3] = tar
            #pc.astype(np.float16)
            
            np.save("/path/to/save/" + i.split('/')[-1].split('.')[0] + '.npy', pc)
            

In [ ]:
tot_grd = glob.glob("/path/to/*.grd")

In [ ]:
from joblib import Parallel, delayed
timeout=99999
Parallel(n_jobs=32, timeout=timeout)(delayed(make_np)(i, dic) for i in tqdm(tot_grd))

In [ ]:
import glob

npy_file = glob.glob("/path/to/npy/saved/*.npy"):


In [ ]:
import json
with open("/path/to/icsd_vol.json", "r") as f:
    dict_vol = json.load(f)

In [ ]:
dict_vol

In [ ]:
from torch_geometric.data import Batch, Data
from tqdm.auto import tqdm
import torch
import torch_geometric.transforms as T
import torch_points3d.core.data_transform as T3D
import pickle
import numpy as np
import glob

In [ ]:
def make_pt(i, dict_vol):
    data = np.load(i)
    pre_transform1 = T.Compose([T3D.GridSampling3D(0.25,  quantize_coords=True, mode="last")])


    vol = dict_vol[i.split('/')[-1].split('.')[0]]
    
    data[:,3] = data[:,3]/vol
    point_cloud = torch.from_numpy(data).to(dtype=torch.float32)
    point_cloud = point_cloud[point_cloud[:, 3]> 0]
    sampled_point_cloud = point_cloud
    pos = sampled_point_cloud[:, :3]
    t = sampled_point_cloud[:, 3:]
    data1 = pre_transform1(Data(pos=pos, x=t))
    torch.save(data1, '/path/to/save/icsd_cart_rand_pt/' + i.split('/')[-1].split('.')[0] + '.pt')

In [ ]:
for i in npy_file:
    make_pt(i, dict_vol)